In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import random
from LQN_utils_state_save_parallel import *
import itertools
from copy import deepcopy 
import sympy as sp
from itertools import groupby

In [2]:
import hashlib

In [11]:
import igraph as ig
import itertools

def EPM_bipartite_graph_generator(num_system, num_ancilla, type): 
    num_total = num_system + num_ancilla

    # Red-Blue 조합 생성 (type=0 가정)
    red_blue_combinations, num_combi = list_all_combinations_with_duplication(num_system, num_ancilla)

    # Ancilla 조합 생성
    ancilla_combinations_pre = generate_combinations(num_total)
    ancilla_combinations = list(itertools.product(ancilla_combinations_pre, repeat=num_ancilla))

    for rb_comb in red_blue_combinations:
        if num_ancilla != 0:
            for bl_comb in ancilla_combinations:
                B = ig.Graph(directed=False)

                # ✅ 노드 추가 (명시적으로 모든 노드 추가)
                B.add_vertices(2 * num_total)

                edges = []
                edge_weights = []

                # ✅ Red-Blue 엣지 추가 (존재하는 노드 ID만 사용)
                for rb_index, vt in enumerate(rb_comb):
                    red = num_total + vt[0]
                    blue = num_total + vt[1]

                    # ✅ 노드 ID 범위 확인
                    if red >= 2 * num_total or blue >= 2 * num_total:
                        print(f"❌ ERROR: Invalid node ID ({red}, {blue})")
                        continue  # 잘못된 노드 ID는 추가하지 않음

                    edges.append((rb_index, red))
                    edge_weights.append(1.0)
                    edges.append((rb_index, blue))
                    edge_weights.append(2.0)

                # ✅ Ancilla 엣지 추가
                for anc_index, vt_list in enumerate(bl_comb):
                    for vt_inx in vt_list:
                        target_node = num_total + vt_inx
                        if target_node >= 2 * num_total:
                            print(f"❌ ERROR: Invalid node ID ({target_node})")
                            continue
                        edges.append((num_system + anc_index, target_node))
                        edge_weights.append(3.0)

                # ✅ Edge List 저장 (Adjacency Matrix 대신)
                B.add_edges(edges)
                B.es["weight"] = edge_weights  

                # ✅ 노드 속성 추가 (category 저장)
                B.vs["category"] = (
                    ["system_nodes"] * num_system + 
                    ["ancilla_nodes"] * num_ancilla + 
                    ["sculpting_nodes"] * (2 * num_total - num_system - num_ancilla)
                )

                yield B  # ✅ iGraph 객체 반환

In [12]:
from scipy.sparse import csr_matrix
import numpy as np

def EPM_digraph_from_EPM_bipartite_graph(B):
    print(f"Before Conversion: Nodes = {B.vs['category']}, Edges = {B.get_edgelist()}")  

    D = ig.Graph(directed=True)

    system_nodes = [v.index for v in B.vs if v["category"] == "system_nodes"]
    ancilla_nodes = [v.index for v in B.vs if v["category"] == "ancilla_nodes"]

    num_system = len(system_nodes)
    num_ancilla = len(ancilla_nodes)
    num_total = num_system + num_ancilla

    # ✅ Edge List → Sparse Matrix 변환 (CSR 형식)
    edge_list = B.get_edgelist()
    weights = B.es["weight"]
    node_count = len(B.vs)

    row_indices = [e[0] for e in edge_list]
    col_indices = [e[1] for e in edge_list]
    data = weights

    adj_matrix_sparse = csr_matrix((data, (row_indices, col_indices)), shape=(node_count, node_count))
    print(f"Sparse Matrix Shape: {adj_matrix_sparse.shape}")

    # ✅ 올바르게 행렬 슬라이싱 (CSR 형식 그대로 사용)
    adj_weight_matrix_D = adj_matrix_sparse[num_total:, :num_total]

    print(f"Extracted Submatrix Shape: {adj_weight_matrix_D.shape}")

    # ✅ 엣지 추가 (Sparse Matrix 활용)
    D.add_vertices(node_count)
    for i, j in zip(*adj_weight_matrix_D.nonzero()):
        weight = adj_weight_matrix_D[i, j]
        D.add_edge(sorted_nodes[j], sorted_nodes[i + num_total], weight=weight)  # ✅ 원래 노드 ID 사용
        print(f"✅ Added Edge: {sorted_nodes[j]} → {sorted_nodes[i + num_total]} (weight={weight})")

    print(f"After Conversion: Nodes = {D.vs['category']}, Edges = {D.get_edgelist()}")

    return D


In [13]:
from collections import defaultdict
import hashlib

def canonical_form_igraph(igraph_graph):
    """
    iGraph 기반 Canonical Form 생성
    """
    perm = igraph_graph.canonical_permutation()
    permuted = igraph_graph.permute_vertices(perm)
    return tuple(map(tuple, permuted.get_adjacency(attribute="weight").data))  # ✅ 가중치 포함 가능

def generate_hash_from_canonical_form(canonical_form):
    """
    Canonical Form을 기반으로 해시 생성
    """
    canonical_str = str(canonical_form)
    return hashlib.sha256(canonical_str.encode('utf-8')).hexdigest()

def process_and_group_by_canonical_form(graph_iter):
    """
    iGraph 기반으로 canonical form을 계산하고 그룹화
    """
    canonical_groups = defaultdict(list)
    for graph in graph_iter:
        canonical_form = canonical_form_igraph(graph)  # ✅ iGraph 기반 canonical form 계산
        canonical_hash = generate_hash_from_canonical_form(canonical_form)

        if canonical_hash not in canonical_groups:
            def graph_generator():
                yield graph  # ✅ iGraph 기반 제너레이터
            canonical_groups[canonical_hash] = graph_generator()

    return canonical_groups  # ✅ 그대로 사용 가능


In [14]:
def is_single_scc(graph):
    sccs = graph.components(mode="strong")
    return len(sccs) == 1 and len(sccs[0]) == len(graph.vs)

def filter_groups_by_scc(grouped_graphs):
    filtered_groups = {}

    for key, graph_gen in grouped_graphs.items():
        try:
            first_graph = next(graph_gen)  # ✅ 제너레이터에서 첫 번째 그래프 가져오기
            D = EPM_digraph_from_EPM_bipartite_graph(first_graph)  # ✅ Sparse Matrix 기반 변환
            if is_single_scc(D):
                filtered_groups[key] = graph_gen  # ✅ 제너레이터 유지
        except StopIteration:
            continue  # ✅ 빈 제너레이터 처리

    return filtered_groups


In [15]:
def extract_unique_bigraphs_with_weights(filtered_groups):
    unique_graphs_list = {}

    for key, graph_gen in filtered_groups.items():
        graph_list = list(graph_gen)  # ✅ 제너레이터를 리스트로 변환
        unique_graphs = []

        for new_graph in graph_list:
            is_unique = True
            for existing_graph in unique_graphs:
                if new_graph.isomorphic_vf2(existing_graph):  # ✅ 구조 비교 먼저 수행
                    # ✅ 가중치까지 비교
                    new_weights = new_graph.es["weight"]
                    existing_weights = existing_graph.es["weight"]

                    if new_weights == existing_weights:  # ✅ 가중치가 동일하면 중복으로 판별
                        is_unique = False
                        break

            if is_unique:
                unique_graphs.append(new_graph)

        unique_graphs_list[key] = unique_graphs  

    return unique_graphs_list  


In [16]:
def visualize_igraph(graph):
    layout = graph.layout("kk")  # Kamada-Kawai 레이아웃
    ig.plot(graph, layout=layout, vertex_label=graph.vs["category"], edge_width=[w for w in graph.es["weight"]])


In [18]:
# 1️⃣ 제너레이터 기반으로 그래프 생성 (iGraph)
graph_generator = EPM_bipartite_graph_generator(num_system=3, num_ancilla=1, type=0)

# 2️⃣ canonical form을 기반으로 그룹화 (iGraph 기반)
grouped_graphs = process_and_group_by_canonical_form(graph_generator)

# 3️⃣ SCC 조건을 만족하는 그래프만 필터링 (iGraph 기반)
filtered_groups = filter_groups_by_scc(grouped_graphs)

# 4️⃣ 중복 제거 (iGraph 기반, isomorphic_vf2() + 가중치 비교 적용)
unique_graphs_list = extract_unique_bigraphs_with_weights(filtered_groups)

# 5️⃣ 최종 결과 출력
for key, unique_graphs in unique_graphs_list.items():
    print(f"Canonical Hash: {key}, Unique Graph Count: {len(unique_graphs)}")


Before Conversion: Nodes = ['system_nodes', 'system_nodes', 'system_nodes', 'ancilla_nodes', 'sculpting_nodes', 'sculpting_nodes', 'sculpting_nodes', 'sculpting_nodes'], Edges = [(0, 7), (0, 6), (1, 7), (1, 6), (2, 7), (2, 6), (3, 4), (3, 5), (3, 6), (3, 7)]
Sparse Matrix Shape: (8, 8)
Extracted Submatrix Shape: (4, 4)


KeyError: 'Attribute does not exist'